
# Reproducing the paper evaluation results

This notebook is the quickest way to reproduce and visualize the evaluation results of the paper.

### Information Captured:
- Emissions produced at each timestep for each seed.
- Abated emissions at each timestep for each seed.
- Net economic output at each timestep for each seed.
- Global and regional temperature variations.
- Savings and emission control rates.

### Plots Included:
- Time series for economic output, emissions, temperature, and abated emissions.
- Regional maps for abated emissions.
- Gini coefficient plots over time.


In [ ]:
# Necessary imports for the notebook

import sys
sys.path.append("../")

import os, glob, pandas as pd

from rl.args import Args
from paper.eval_utils import *
from eval import *

# Enable inline plotting
%matplotlib inline
!pip install --upgrade nbformat

os.chdir('..') #Make sure your current working directory is the root folder of the project


In [3]:
# Define the paths to the policies and the baseline emissions data to used to compute abated emissions
CURRENT_WORKING_DIR = os.getcwd()
PAPER_FOLDER = os.path.join(CURRENT_WORKING_DIR, "paper")
POLICY_FOLDER = os.path.join(PAPER_FOLDER, "paper_policies") # Folder containing the checkpoints and pickles
ALL_POLICIES = glob.glob(os.path.join(POLICY_FOLDER, "*"))

baselines_emissions_data_path = os.path.join(PAPER_FOLDER, "baseline_emissions.csv") # Used to calculate abated emissions

# Load the baseline emissions data 
base_emissions = pd.read_csv(baselines_emissions_data_path, delimiter=";").iloc[:, 1:].to_numpy()


In [ ]:
# Running evaluation for the 3 chosen policies across 10 evaluation seeds each

evaluation_seeds = range(1,11) # Seeds 1 to 10
global_data_dict = {}

for policy_base_path in ALL_POLICIES:

    policy_folder_name = os.path.basename(policy_base_path)
    print(f"Running evaluations for policy {policy_folder_name}")
    
    checkpoint_path = glob.glob(os.path.join(policy_base_path, "checkpoints", "*"))[0]

    loaded_args = Args()
    loaded_args.network_model_config = glob.glob("rl/nn/params/mlp.json")[0]

    for eval_seed in evaluation_seeds:
        
        print(f"Running evaluation for seed={str(eval_seed)}")
        
        seed_evaluation_data = evaluate_seed(base_emissions=base_emissions, loaded_args=loaded_args, checkpoint_path=checkpoint_path, eval_seed=eval_seed)
        
        seed_evaluation_dict = {str(eval_seed): seed_evaluation_data}
        
        if policy_folder_name not in global_data_dict.keys():
            global_data_dict[policy_folder_name] = seed_evaluation_dict
        else:
            global_data_dict[policy_folder_name].update(seed_evaluation_dict)


In [5]:
# AGGREGATE DATA ACROSS SEEDS

evaluation_data = global_data_dict
aggregated_data = aggregate_data_across_seeds(evaluation_data=evaluation_data)

(
    rice_abated_emissions_data,
    rice_emissions_data,
    rice_global_temperature_data,
    rice_net_economic_output_data,
) = load_rice_data(current_working_dir=PAPER_FOLDER)


In [ ]:
# PLOT TIME SERIES  

plot_economic_output_over_time(
    aggregated_data=aggregated_data, rice_data=rice_net_economic_output_data, current_working_dir=PAPER_FOLDER, mode="show"
)
plot_total_emissions_over_time(
    aggregated_data=aggregated_data, rice_data=rice_emissions_data, current_working_dir=PAPER_FOLDER, mode="show"
)
plot_average_global_temperature_over_time(
    aggregated_data=aggregated_data, rice_data=rice_global_temperature_data, current_working_dir=PAPER_FOLDER, mode="show"
)
plot_total_abated_emissions_over_time(
    aggregated_data=aggregated_data, rice_data=rice_abated_emissions_data, current_working_dir=PAPER_FOLDER, mode="show"
)


In [ ]:
# PLOT REGIONAL MAPS

abated_emissions_data = {
    policy_name: policy_data["mean"]["abated_emissions"]
    for policy_name, policy_data in aggregated_data.items()
}

abated_emissions_data["rice"] = rice_abated_emissions_data

plot_regional_maps(abated_emissions_data, current_working_dir=PAPER_FOLDER, mode="show")


In [ ]:
# PLOT GINI COEFFICIENTS

plot_gini_over_time(
    aggregated_data=aggregated_data,
    rice_data={
        "net_economic_output": rice_net_economic_output_data,
        "emissions": rice_emissions_data,
        "abated_emissions": rice_abated_emissions_data,
    },
    current_working_dir=CURRENT_WORKING_DIR,
    mode="show",
    moving_average_n=5
)
